In [3]:
import pandas as pd 
import numpy as np 
import re 

In [4]:
df=pd.read_csv('BestSellPhone_TGDD.csv')

# Clean raw Data

In [5]:
def clean_utility(row):
    return ' '.join(row.replace('\n','').strip(' ').split())
df['phone_utility']=df['phone_utility'].apply(clean_utility)

def clean_screen(row):
    _=row.replace('\n','').strip().split()
    return ' '.join(_)
df['phone_screen']=df['phone_screen'].apply(clean_screen)

def clean_rating(row):
    _=row.replace('\n','').strip().split()
    return ' '.join(_)
df['phone_rating']=df['phone_rating'].apply(clean_rating)

def enrich_vote_rating_raw(row):
    _=row.split()[0]
    return _ 
df['vote_rating'] = df['phone_rating'].apply(enrich_vote_rating_raw)

def enrich_sold_quantity_raw(row):
    _=row.split()
    if len(_) > 3:
        raw_sold_quantity=_[4]
        return raw_sold_quantity
    else:
        return 0
df['sold_quantity'] = df['phone_rating'].apply(enrich_sold_quantity_raw)

df.loc[df['vote_rating'].str.isalpha(),'vote_rating'] = 0

def enrich_sold_quantity(row):
    if str(row).find('k') > 0:
        _ = int(row.replace(',','').replace('k','')) * 1000
        return _
    else:
        return row
df['sold_quantity']=df['sold_quantity'].apply(enrich_sold_quantity)

def clean_phone_screen(row):
    return row.replace('"','').strip()
df['phone_screen']=df['phone_screen'].apply(clean_phone_screen)

# Transfrom Datatype

In [6]:
df['phone_price']=df['phone_price'].astype('int64')
df['sold_quantity']=df['sold_quantity'].astype('int64')
df['vote_rating']=df['vote_rating'].astype('float64')

# Remove oulier

In [7]:
df_2=df.copy()
df_2=df_2[df_2['sold_quantity']>0]

# Enrich Data

In [8]:
df_2['gross_revenue']=df_2['sold_quantity'] * df_2['phone_price']

In [9]:
def price_segment(row):
    if row < 10000000:
        return '0 to 10M VND'
    elif row < 20000000:
        return '10M to 20M VND'
    else:
        return 'Upper 20MVND'
df_2['price_segment']=df_2['phone_price'].apply(price_segment)

In [10]:
def return_size_screen_phone(row):
    list_row=row.split()
    leght_list=len(list_row)
    if leght_list < 2:
        return ' '.join(list_row)
    elif leght_list < 5:
        return list_row[-1]
    else:
        return ' '.join(list_row[list_row.index('Chính'):])
df_2['size_screen_inch']=df_2['phone_screen'].apply(return_size_screen_phone)

In [11]:
def return_screen_phone(row):
    list_row=row.split()
    leght_list=len(list_row)
    if leght_list < 2:
        return ' '.join(list_row)
    elif leght_list < 5:
        return ' '.join(list_row[:-1])
    else:
        return ' '.join(list_row[:list_row.index('Chính')])
def detail_none_screen(row):
    _ = row.replace('.','')
    if _.isdigit():
        return 'NaN'
    else:
        return row
df_2['info_screen']=df_2['phone_screen'].apply(return_screen_phone).apply(detail_none_screen)

In [12]:
def rating_segment(row):
    if 0 < row < 3:
        return 'Poor'
    elif 3 < row < 4.5:
        return 'Average'
    else:
        return 'Excellent'
df_2['rating_segment'] = df_2['vote_rating'].apply(rating_segment)

In [13]:
def return_ram(row):
    list_row=row.split()
    if 'RAM:' in list_row and 'lượng:' in list_row:
        ram_idx=list_row.index('RAM:')
        return list_row[ram_idx+1]
    else:
        return 'NaN'
def return_rom(row):
    list_row=row.split()
    if 'RAM:' in list_row and 'lượng:' in list_row:
        rom_idx=list_row.index('lượng:')
        return list_row[rom_idx+1]
    else:
        return 'NaN'
df_2['RAM(GB)'] = df_2['phone_utility'].apply(return_ram)
df_2['ROM(GB)'] = df_2['phone_utility'].apply(return_rom)

In [14]:
def return_battery(row):
    list_row=row.split()
    if 'Pin' in list_row:
        pin_idx=list_row.index('Pin')
        return ' '.join(list_row[pin_idx+1:pin_idx+3]).replace(',','')
    else:
        return 'NaN'
df_2['battery']=df_2['phone_utility'].apply(return_battery)

In [20]:
df_2['phone_price_Milion_VND'] = df_2['phone_price'] / 1000000

In [22]:
df_2['gross_revenue_Bilion_VND'] = df_2['gross_revenue'] / 1000000000

In [23]:
df_2

,phone_id,phone_name,phone_price,phone_brand,category,phone_img,phone_screen,phone_utility,phone_rating,vote_rating,...,gross_revenue,price_segment,size_screen_inch,info_screen,rating_segment,RAM(GB),ROM(GB),battery,phone_price_Milion_VND,gross_revenue_Bilion_VND
0,329149,Điện thoại iPhone 16 Pro Max 256GB,31390000,iPhone (Apple),Điện thoại,https://cdn.tgdd.vn/Products/Images/42/329149/...,Super Retina XDR 6.9,Chip Apple A18 Pro 6 nhân RAM: 8 GB Dung lượng...,4.9 • Đã bán 98k So sánh,4.9,...,3076220000000,Upper 20MVND,6.9,Super Retina XDR,Excellent,8,256,33 giờ,31.39,3076.22000
1,333147,Điện thoại Xiaomi Redmi Note 14 8GB/128GB,5490000,Xiaomi,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/333147/...,Full HD+ 6.67,Chip MediaTek Helio G99-Ultra 8 nhân RAM: 8 GB...,"4.9 • Đã bán 25,1k So sánh",4.9,...,1377990000000,0 to 10M VND,6.67,Full HD+,Excellent,8,128,5500 mAh,5.49,1377.99000
2,325800,Điện thoại Xiaomi Redmi 13 6GB/128GB,3290000,Xiaomi,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/325800/...,Full HD+ 6.79,Chip MediaTek Helio G91 Ultra 8 nhân RAM: 6 GB...,"4.9 • Đã bán 144,2k So sánh",4.9,...,4744180000000,0 to 10M VND,6.79,Full HD+,Excellent,6,128,5030 mAh,3.29,4744.18000
3,331204,Điện thoại Samsung Galaxy A16 5G 8GB/256GB,6490000,Samsung,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/331204/...,Full HD+ 6.7,Chip MediaTek Dimensity 6300 5G 8 nhân RAM: 8 ...,"4.9 • Đã bán 35,1k So sánh",4.9,...,2277990000000,0 to 10M VND,6.7,Full HD+,Excellent,8,256,5000 mAh,6.49,2277.99000
4,332936,Điện thoại OPPO Reno13 F 5G 8GB/256GB,9990000,OPPO,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/332936/...,Full HD+ 6.67,Chip Snapdragon 6 Gen 1 5G 8 nhân RAM: 8 GB Du...,"4.9 • Đã bán 24,6k So sánh",4.9,...,2457540000000,0 to 10M VND,6.67,Full HD+,Excellent,8,256,5800 mAh,9.99,2457.54000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,333281,Điện thoại realme Note 60x 4GB/64GB,2990000,realme,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/333281/...,HD+ 6.74,Chip Unisoc Tiger T612 RAM: 4 GB Dung lượng: 6...,5 • Đã bán 3k So sánh,5.0,...,8970000000,0 to 10M VND,6.74,HD+,Excellent,4,64,5000 mAh,2.99,8.97000
100,327975,Điện thoại realme C61 4GB/128GB,3290000,realme,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/327975/...,HD+ 6.745,Chip Unisoc Tiger T612 RAM: 4 GB Dung lượng: 1...,5 • Đã bán 105 So sánh,5.0,...,345450000,0 to 10M VND,6.745,HD+,Excellent,4,128,5000 mAh,3.29,0.34545
101,302953,Điện thoại OPPO Find N3 5G 16GB/512GB,38990000,OPPO,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/302953/...,Quad HD+ (2K+) Chính 7.82 & Phụ 6.31,Chip Snapdragon 8 Gen 2 8 nhân RAM: 16 GB Dung...,5 • Đã bán 374 So sánh,5.0,...,14582260000,Upper 20MVND,Chính 7.82 & Phụ 6.31,Quad HD+ (2K+),Excellent,16,512,4805 mAh,38.99,14.58226
102,311354,Điện thoại OPPO A58 8GB/128GB,4990000,OPPO,Điện thoại,https://cdn.tgdd.vn/Products/Images/42/311354/...,Full HD+ 6.72,Chip MediaTek Helio G85 RAM: 8 GB Dung lượng: ...,"4.9 • Đã bán 123,5k So sánh",4.9,...,6162650000000,0 to 10M VND,6.72,Full HD+,Excellent,8,128,5000 mAh,4.99,6162.65000


In [25]:
df_3=df_2.drop(['phone_rating','phone_screen','category','phone_id'],axis=1)

In [26]:
df_3=df_3[['phone_name','phone_brand','phone_price','phone_price_Milion_VND','phone_img','phone_utility','RAM(GB)','ROM(GB)','battery','info_screen','size_screen_inch','vote_rating','price_segment','rating_segment','sold_quantity','gross_revenue','gross_revenue_Bilion_VND']]

In [27]:
df_3.to_csv('Best_sell_phone_cleaned.csv',index=False)